In [ ]:
# Analyse des consommations électriques régionales en France de 2013 à 2022 sur la base des données de RTE stockées dans
# le fichier eco2mix-regional-cons-def.csv
# Exploration des données brutes du fichier (recherche des valeurs manquantes, colonnes utiles/inutiles, types des données, brèves statitistiques...
# Thierry ALLEM - Projet DataScientest - Formation Data Analyst -2024


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import warnings

# Pour ignorer tous les warnings
warnings.filterwarnings('ignore')


In [ ]:
# Ajustement  de la largeur des colonnes des dataframes
pd.set_option('display.max_columns', None)  # afficher toutes les colonnes
pd.set_option('display.width', None)        # éviter le retour à la ligne auto
pd.set_option('display.colheader_justify', 'left')  # alignement des en-têtes

In [ ]:
df_brut = pd.read_csv('eco2mix-regional-cons-def.csv',sep=';',na_values='ND',low_memory=False)

In [ ]:
df_brut['Code INSEE région'].unique()

In [ ]:
df_brut.tail(20)

In [ ]:
# Affichage du nombre de lignes et colonnes du dataframe et décompte des types des valeurs

print("Nombre de lignes du dataframe : ", len(df_brut.index))
print("Nombre de colonnes du dataframe : ", len(df_brut.columns))
display("Types de valeurs :",df_brut.dtypes.value_counts() )
print("Nombre de doublons : ",df_brut.duplicated().sum())

In [ ]:
df_brut.info()

In [ ]:
# Brèves statistiques de la colonne 'Consommation (MW)'
df_brut['Consommation (MW)'].describe()

In [ ]:
# Création d'un dataframe de description des colonnes de df_brut, sans regroupement par région
df_colonnes = pd.DataFrame()

# Stockage des noms des colonnes du dataframe df_brut
df_colonnes['Nom_colonne'] = list(df_brut.columns)

# Attribution du type de valeurs des colonnes
df_colonnes['Type_valeurs'] = df_brut.dtypes.values

# Ajout d'une colonne 'Valeurs_uniques' par défaut à 'Valeurs continues'
df_colonnes['Valeurs_uniques'] = 'Valeurs continues'

# Spécificité pour certaines colonnes : affichage des valeurs uniques
df_colonnes.loc[df_colonnes['Nom_colonne'] == 'Code INSEE région', 'Valeurs_uniques'] = str(df_brut['Code INSEE région'].unique())
df_colonnes.loc[df_colonnes['Nom_colonne'] == 'Région', 'Valeurs_uniques'] = str(df_brut['Région'].unique())
df_colonnes.loc[df_colonnes['Nom_colonne'] == 'Nature', 'Valeurs_uniques'] = str(df_brut['Nature'].unique())

# Option d'affichage pour éviter la troncature des chaînes
pd.set_option('display.max_colwidth', None)

# Calcul du nombre de valeurs non nulles
df_colonnes['Nb_valeurs'] = df_brut.count().values

# Calcul du nombre de NaN
df_colonnes['nb_NaN'] = df_brut.isna().sum().values

# Total des données attendues
df_colonnes['Nb_datas'] = df_colonnes['Nb_valeurs'] + df_colonnes['nb_NaN']

# Pourcentage de données présentes et manquantes
df_colonnes['%_Valeurs'] = round((df_colonnes['Nb_valeurs'] / df_colonnes['Nb_datas']) * 100, 2)
df_colonnes['%_NaN'] = round((df_colonnes['nb_NaN'] / df_colonnes['Nb_datas']) * 100, 2)

# Identification des colonnes numériques uniquement (à partir de la 7ème colonne en général)
selected_columns = df_brut.columns[6:]
numeric_columns = [col for col in selected_columns if pd.api.types.is_numeric_dtype(df_brut[col])]

# Initialisation des colonnes pour le comptage
df_colonnes['Valeurs_POSITIVES'] = 0
df_colonnes['Valeurs_NULLES'] = 0
df_colonnes['Valeurs_NEGATIVES'] = 0

# Boucle pour calculer le nombre de valeurs positives, nulles et négatives uniquement sur les colonnes numériques
for col in numeric_columns:
    df_colonnes.loc[df_colonnes['Nom_colonne'] == col, 'Valeurs_POSITIVES'] = (df_brut[col] > 0).sum()
    df_colonnes.loc[df_colonnes['Nom_colonne'] == col, 'Valeurs_NULLES'] = (df_brut[col] == 0).sum()
    df_colonnes.loc[df_colonnes['Nom_colonne'] == col, 'Valeurs_NEGATIVES'] = (df_brut[col] < 0).sum()

# Calcul des pourcentages correspondants
df_colonnes['%_valeurs_POSITIVES'] = round((df_colonnes['Valeurs_POSITIVES'] / df_colonnes['Nb_valeurs']) * 100, 2)
df_colonnes['%_valeurs_NULLES'] = round((df_colonnes['Valeurs_NULLES'] / df_colonnes['Nb_valeurs']) * 100, 2)
df_colonnes['%_valeurs_NEGATIVES'] = round((df_colonnes['Valeurs_NEGATIVES'] / df_colonnes['Nb_valeurs']) * 100, 2)

# Vérification de la cohérence des taux (somme des pourcentages)
df_colonnes['Cumul_taux_valeurs'] = df_colonnes['%_valeurs_POSITIVES'] + df_colonnes['%_valeurs_NEGATIVES'] + df_colonnes['%_valeurs_NULLES']

# Désignation de la colonne 'Nom_colonne' comme index
df_colonnes.set_index('Nom_colonne', inplace=True)

# Export du compte-rendu dans un fichier Excel
file_export2 = 'Rapp_explo_col_eco2mix.xlsx'
df_colonnes.to_excel(file_export2)

# Message de confirmation
display("Rapport préliminaire d'exploration des colonnes du dataframe 'eco2mix-regional-cons-def' exporté avec succès dans le fichier Excel 'Rapp_explo_col_eco2mix.xlsx'")



In [ ]:
df_colonnes.info()

In [ ]:
display(df_colonnes)

In [ ]:
df_colonnes = df_colonnes.reset_index()  # Réinitialisation de l'index

In [ ]:
df_colonnes.info()

In [ ]:
# Recherche de la date la plus ancienne des "Données définitives" et celle des "Données consolidées"  par région

df_donnees=df_brut.copy()

# Groupement par région et nature
grouped = df_donnees.groupby(['Région','Nature'])['Date - Heure']

# Obtention de la dernière ligne pour 'Données définitives' et la première ligne pour 'Données consolidées'
result = pd.concat([grouped.last(), grouped.first()])

# Enregistrement des résultats dans un fichier Excel
result.to_excel('resultats.xlsx', index=True)

print("Résultats enregistrés dans 'resultats.xlsx'")


In [ ]:
display(result)

In [ ]:
# Histogrammes  Valeurs/Nan et distribution des valeurs de df_brut, sans regroupement par région

# Suppression de colonnes  et de lignes de df_colonnes pour affichage d'histogrammes ultérieurs
# Liste des colonnes à supprimer
cols_to_drop = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13,14]

# Suppression des colonnes
df_hist_val_nan = df_colonnes.drop(df_colonnes.columns[cols_to_drop], axis=1)

# Réinitialisation de l'index
df_hist_val_nan = df_hist_val_nan.reset_index(drop=True)

# Liste des lignes à supprimer
rows_to_drop = [0,1, 2, 3, 4, 5,31]

# Suppression des lignes
df_hist_val_nan = df_hist_val_nan.drop(rows_to_drop, axis=0)

# Mise en forme longue du DataFrame
df_melt1 = df_hist_val_nan.melt(id_vars='Nom_colonne', var_name='Categorie', value_name='Proportion')

# Création d'une figure et d'un axe
fig, axs = plt.subplots(2,1, figsize=(50, 30))

# Création du diagramme à barres groupées
width = 0.35
histo_colonnes = sns.barplot(x='Nom_colonne', y='Proportion', hue='Categorie', data=df_melt1, ax=axs[0])
#histo_colonnes = sns.barplot(x='Nom_colonne', y='Value', hue='Category', data=df_melt1, ax=ax)

# Affichage d'une grille
plt.grid(linestyle = '-', axis = 'y')

# Récupération des positions x actuelles sur l'axe des abscisses

x_separation = histo_colonnes.get_xticks()

# Ajout d'une ligne verticale après chaque barre

for separation  in x_separation:
    histo_colonnes.axvline(separation + 0.5, color='red', linestyle='-')

# Affichage du titre
plt.title('Répartion des valeurs présentes et manquantes')

# Définition des limites de l'axe des y
plt.ylim(0, 100)

# Sauvegarde du graphique dans un fichier
histo_colonnes.figure.savefig('Histogramme_valeurs_nan.png')

# Histogramme des valeurs présentes- Répartitions

# Suppression de colonnes  et de lignes de df_colonnes pour affichage d'histogrammes ultérieurs
# Liste des colonnes à supprimer
cols_to_drop = [1, 2, 3, 4, 5, 6, 7, 8, 10, 12,14]

# Suppression des colonnes
df_hist_val = df_colonnes.drop(df_colonnes.columns[cols_to_drop], axis=1)

# Réinitialisation de l'index
df_hist_val = df_hist_val.reset_index(drop=True)

# Liste des lignes à supprimer
rows_to_drop = [0, 2, 3, 4, 5,31]

# Suppression des lignes
df_hist_val = df_hist_val.drop(rows_to_drop, axis=0)

# Mise en forme longue du DataFrame
df_melt2 = df_hist_val.melt(id_vars='Nom_colonne', var_name='Categorie', value_name='Proportion')

# Création du diagramme à barres groupées
width = 0.35
palette_colors = palette_colors = {
    'Valeurs_NULLES': 'green',        # Pour les valeurs nulles
    '%_valeurs_POSITIVES': '#3274a1',  # Pour les valeurs positives
    '%_valeurs_NEGATIVES': 'brown'     # Pour les valeurs négatives
}
histo_colonnes2 = sns.barplot(x='Nom_colonne', y='Proportion', hue='Categorie', palette =  palette_colors, data=df_melt2, ax=axs[1])
#histo_colonnes2 = sns.barplot(x='Nom_colonne', y='Pourcentage', hue='Distribution des valeurs', data=df_melt2, ax=axs[1])

# Affichage d'une grille
plt.grid(axis = 'y', linestyle = '--')

# Affichage du titre
plt.title('Répartion des valeurs selon leur signe algébrique')

# Définition des limites de l'axe des y
plt.ylim(0, 100)

# Récupération des positions x actuelles sur l'axe des abscisses

x_separation = histo_colonnes2.get_xticks()

# Ajout d'une ligne verticale après chaque barre

for separation  in x_separation:
    histo_colonnes2.axvline(separation + 0.5, color='red', linestyle='-')

# Affichage du graphique
plt.show()

# Sauvegarde du graphique dans un fichier
histo_colonnes2.figure.savefig('Repartition_valeurs_col.png')



In [ ]:
# Affichage des colonnes
print(df_colonnes.iloc[:, 0].unique())

In [ ]:
# Export du compte-rendu des colonnes dans un fichier Excel

file_export2 ='Rapp_explo_col_eco2mix.xlsx'

df_colonnes.to_excel(file_export2)

display("Rapport préliminaire d'exploration des colonnes du dataframe 'eco2mix-regional-cons-def' exporté avec succès dans le fichier Excel 'Rapp_explo_col_eco2mix.xlsx'")

In [ ]:
display(df_colonnes)

In [ ]:
df_colonnes.info()

In [ ]:
# Création d'une copie du dataframe d'origine avant modifications

df=df_brut.copy()
df.head()

In [ ]:
df.info()

In [ ]:
# Création d'une nouvelle colonne de dates avec le format '%Y-%m-%d %H:%M'
df.insert(5, 'date_heure_cet', (df['Date'].astype(str) + ' ' + df['Heure'].astype(str)))


In [ ]:
df.head()

In [ ]:
# Conversion des colonnes 'date','Heure' et 'date_heure_cet' au format datetime
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d', yearfirst = True, exact = True)
df['Heure'] = pd.to_datetime(df['Heure'], format = '%H:%M')
df['date_heure_cet'] = pd.to_datetime(df['date_heure_cet'], format='%Y-%m-%d %H:%M',  errors='coerce')
df.dtypes

In [ ]:
# Ajout d'une colonne 'Mois' et d'une colonne 'Année' extraites de la colonne 'Date'
df.insert(4, 'Mois', df['Date'].dt.month)

df.insert(5, 'Année', df['Date'].dt.year)

# Création d'une colonne date au format YYYY-MM, type Object

df.insert(6, 'Date YM', df['Date'].dt.strftime('%Y-%m'))


In [ ]:
df.head()

In [ ]:
# Suppression de la colonne 'Column 30'

df.drop(columns=['Column 30'], inplace=True)

In [ ]:
# Export du dataframe dans un fichier csv

eco2mix_temporaire = df.copy()
file_export_csv ='eco2mix_temporaire.csv'

eco2mix_temporaire.to_csv(file_export_csv,sep=';',encoding='latin-1')


In [ ]:
# Calcul des statistiques descriptives pour la colonne 'Consommation (MW)'
stats = df['Consommation (MW)'].describe()

# Conversion des statistiques en DataFrame et transposition des résultats
stats_df = stats.to_frame().T

display(stats_df)

In [ ]:
# Filtrage des années entre 2013 et 2022
df_filtered = df[(df['Année'] >= 2013) & (df['Année'] <= 2021)]

# Groupement par région et application de describe()
result = df_filtered.groupby('Région')['Consommation (MW)'].describe()

# Calcul des statistiques descriptives pour l'ensemble des régions et années de 2013 à 2021
#stats = df['Consommation (MW)'].describe()
# Conversion des statistiques en DataFrame et transposition des résultats
# national_stats = stats.to_frame().T

# Créeation d'une nouvelle ligne dans le tableau incluant les statitistiques nationales de 2013 à 2021
# df_result = pd.concat([national_stats, result], axis =1)

display(result)

In [ ]:
# 1.2 : Gestion des noms et types de S NOMS ET TYPES DE VARIABLES

# on fait un premier dictionnaire pour changer les en-têtes du df_2 dans la même version que le df_1
dictionnaire_entetes1 = {'Code INSEE région' : 'code_insee_region',
                'Région' : 'region',
                'Nature' : 'nature',
                'Date' : 'date',
                'Heure' : 'heure',
                'Date - Heure' : 'date_heure',
                'Mois' : 'mois',
                'Année' :'annee',
                'Date YM' : 'date_ym',
                'Consommation (MW)' : 'consommation',
                'Thermique (MW)' : 'thermique',
                'Nucléaire (MW)' : 'nucleaire',
                'Eolien (MW)' : 'eolien',
                'Solaire (MW)' : 'solaire',
                'Hydraulique (MW)' : 'hydraulique',
                'Pompage (MW)' : 'pompage',
                'Bioénergies (MW)' : 'bioenergies',
                'Ech. physiques (MW)' : 'ech_physiques',
                'Stockage batterie' : 'stockage_batterie',
                'Déstockage batterie' : 'destockage_batterie',
                'TCO Thermique (%)' : 'tco_thermique',
                'TCH Thermique (%)' : 'tch_thermique',
                'TCO Nucléaire (%)' : 'tco_nucleaire',
                'TCH Nucléaire (%)' : 'tch_nucleaire',
                'TCO Eolien (%)' : 'tco_eolien',
                'TCH Eolien (%)' : 'tch_eolien',
                'TCO Solaire (%)' : 'tco_solaire',
                'TCH Solaire (%)' : 'tch_solaire',
                'TCO Hydraulique (%)' : 'tco_hydraulique',
                'TCH Hydraulique (%)' : 'tch_hydraulique',
                'TCO Bioénergies (%)' : 'tco_bioenergies',
                'TCH Bioénergies (%)' : 'tch_bioenergies'}
df=df.rename(dictionnaire_entetes1, axis = 1)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#Création de boxplot de la colonne "Consommation" à l'échelle métropolitaine et par année de 2013 à 2021 (années complètes)
from matplotlib.ticker import FuncFormatter

# Fonction pour formater les étiquettes des ordonnées avec des séparateurs de milliers
def thousands_formatter(x, pos):
    return f'{int(x):,}'.replace(',', ' ')
    
# Filtrage des données pour les années 2013 à 2021
df_metropole = df[(df['annee'] >= 2013) & (df['annee'] <= 2021)]

# Regroupement par 'date_heure' pour obtenir la consommation totale de toutes les régions à chaque instant
df_metropole_grouped = df_metropole.groupby(['date_heure', 'annee'], as_index=False)['consommation'].sum()

# Configuration du style de seaborn
sns.set_theme(palette="bright")

# Création de la figure pour le boxplot
plt.figure(figsize=(12, 8))

# Création du boxplot
sns.boxplot(x='annee', y='consommation', data=df_metropole_grouped, color='fuchsia')

# Ajout de la grille verticale
plt.grid(axis='x', linestyle='--', color='gray')

# Graduation de l'axe des ordonnées tous les 5000 MW
plt.yticks(range(0, int(df_metropole_grouped['consommation'].max()) + 5000, 5000))
# Formater les étiquettes des ordonnées avec des séparateurs de milliers
plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

plt.ylabel('Année')
plt.ylabel('Consommation (MW)')

# Définition du titre du graphique
plt.title('Boxplots de la consommation de la Métropole (2013-2021)')

# Sauvegarde du graphique dans un fichier image
plt.savefig('metropole_boxplot.png',dpi =300)

# Affichage du graphique
plt.show()



In [ ]:
#Création de boxplot de la colonne "Consommation" à l'échelle métropolitaine et par année de 2013 à 2021 (années complètes)

# Liste des régions

sns.set_theme(palette="bright")

df_metropole = df[(df['annee'] >= 2013) & (df['annee'] <= 2021)]
    
plt.figure(figsize=(10, 6))

# Création du boxplot
sns.boxplot(x='annee', y='consommation', data=df_metropole, color = 'purple')

# Bornage de l'axe des abscisses
#plt.ylim(0, 16000)
      
 # Définition du titre du graphique
plt.title('Boxplots de la consommation de la Métropole')

    # Sauvegarde du graphique dans un fichier image
plt.savefig('metropole' + '_boxplot.png')

plt.show()

In [ ]:
#Création de boxplot de la colonne "Consommation" par région et par année de 2013 à 2021 (années complètes)

# Liste des régions
regions = df['region'].unique()
sns.set_theme(palette="bright")

for region in regions:
    # Création d'un sous-dataframe pour chaque région
    df_region = df[(df['region'] == region) & (df['annee'] >= 2013) & (df['annee'] <= 2021)]
    
    # Création d'une figure pour chaque région
    plt.figure(figsize=(10, 6))

    # Création du boxplot
    sns.boxplot(x='annee', y='consommation', data=df_region)

    # Uniformisation de l'axe des abscisses
    plt.ylim(0, 16000)
      
    # Définition du titre du graphique
    plt.title('Boxplots de la consommation pour la région ' + region)

    # Sauvegarde du graphique dans un fichier image
    plt.savefig(region + '_boxplot.png')

plt.show()


In [ ]:
df.info()

In [ ]:
# ****** ETUDE DES VALEURS PRESENTES ******

# Calcul du nombre de valeurs présentes pour chaque colonne, par région et par mois.

count_val = df.groupby([df['region'], df['date_ym']],as_index=False).count()

# Suppression des colonnes  de count_val

count_val.drop(columns = count_val.columns[2:9], inplace=True)

# AFFICHAGE DES HEATMAPS DES VALEURS PRESENTES, REGROUPEES PAR REGION ET PAR MOIS

# 1) Création d'un dictionnaire associant les colonnes mappées avec les noms des graphiques et des fichiers images à créer

dict_heatmaps_energie = {}
colonnes=['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
c= 0
for i in range(2, 27):
    nom_colonne = count_val.columns[i]
    pivot = 'pivot_'+ nom_colonne[:9]
    nom_heatmap = nom_colonne[:9].replace(' ', '_') + 'val_map'
    nom_fichier = colonnes[c]+'_'+ nom_colonne[:9].replace(' ', '_') + '_val_heatmap.png'
    dict_heatmaps_energie[nom_colonne] = (pivot, nom_heatmap, nom_fichier)
    c+=1

display(dict_heatmaps_energie)

# 2) Affichage des chronogramme des valeurs présentes pour chaque colonne de données d'énergie

for i in range(2,27):
    
    nom_colonne = count_val.columns[i]
    pivot, nom_heatmap, nom_fichier = dict_heatmaps_energie[nom_colonne]
                                                       
    # Pivot du DataFrame pour obtenir les dates comme colonnes
 
    pivot_df = count_val.pivot(index='region', columns='date_ym', values=nom_colonne)

    # Remplissage des valeurs manquantes avec 0
    pivot_df.fillna(0, inplace=True)

    # Création d'un heatmap
    plt.figure(figsize=(100, 10))
    nom_heatmap = sns.heatmap(pivot_df, cmap='viridis', annot=True, fmt='d', cbar_kws={'label': 'Nombre valeurs présentes'},
                                 linewidths = 1, vmin = 0, vmax = 1488)
    plt.title(f"Nombre de données présentes pour la colonne {nom_colonne}, par région et mois", fontsize = 16, pad = 20)
    plt.xlabel('date_ym', fontsize = 16)
    plt.ylabel('region', fontsize = 16)
    plt.show()

    nom_heatmap.figure.savefig(nom_fichier)


# Export Excel du compte-rendu du nb de valeurs présentes par colonne après regroupement region/jour-mois

file_export_count_val ='Rapp_count_val_eco2mix.xlsx'

count_val.to_excel(file_export_count_val)

print("Rapport préliminaire de décompte des valeurs après regroupements exporté avec succès dans le fichier Excel 'Rapp_count_val_eco2mix.xlsx'")



In [ ]:
count_val.tail(30)

In [ ]:
# Nettoyage des noms de colonnes
df.columns = df.columns.str.strip()

# Vérification
print(df.columns)

In [ ]:
# Calcul du nombre de valeurs manquantes par région et mois
count_nan = df.groupby(['region', 'date_ym']).agg(lambda x: x.isnull().sum()).reset_index()

# Vérification des colonnes disponibles
print("Colonnes après groupby et reset_index :", count_nan.columns.tolist())

# Filtrage de certaines colonnes spécifiques
# Conserver uniquement 'region', 'date_ym' + colonnes 10 à 34
cols_a_garder = ['region', 'date_ym'] + count_nan.columns[10:35].tolist()
count_nan_filtre = count_nan[cols_a_garder]

#  Export Excel du compte-rendu complet
file_export_count_nan = 'Rapp_count_nan_eco2mix.xlsx'
count_nan.to_excel(file_export_count_nan, index=False)
print(f"✅ Rapport complet exporté : {file_export_count_nan}")

# Export d'une version filtrée
# Les colonnes énergie sont : 'nucleaire', 'thermique', etc.
colonnes_energie = ['region', 'date_ym'] + [col for col in count_nan.columns if 'energie' in col.lower()]
count_nan_filtre = count_nan[colonnes_energie]
file_exp_count_nan_filtre = 'Rapp_count_nan_eco2mix_FILTRE.xlsx'
count_nan_filtre.to_excel(file_exp_count_nan_filtre, index=False)
print(f"✅ Rapport filtré exporté : {file_exp_count_nan_filtre}")

# Préparation du dictionnaire pour les heatmaps
dict_heatmaps_energie_nan = {}
for i in range(2, len(count_nan.columns)):
    nom_colonne = count_nan.columns[i]
    pivot = f'pivot_{nom_colonne[:7]}'
    nom_heatmap = nom_colonne[:7].replace(' ', '_') + '_nan_map'
    nom_fichier = nom_colonne[:7].replace(' ', '_') + '_nan_heatmap.png'
    dict_heatmaps_energie_nan[nom_colonne] = (pivot, nom_heatmap, nom_fichier)

print("🗺️ Dictionnaire des heatmaps prêt :", dict_heatmaps_energie_nan)

# Affichage et export des heatmaps
for nom_colonne in list(dict_heatmaps_energie_nan.keys()):
    pivot, nom_heatmap, nom_fichier = dict_heatmaps_energie_nan[nom_colonne]

    # Création du pivot (région en ligne, mois en colonne)
    pivot_df = count_nan.pivot(index='region', columns='date_ym', values=nom_colonne)

    # Remplacement des NaN par 0
    pivot_df.fillna(0, inplace=True)

    # Affichage
    plt.figure(figsize=(30, 8))
    heatmap = sns.heatmap(pivot_df, cmap='YlOrRd', annot=True, fmt='.0f',
                          cbar_kws={'label': 'Nombre valeurs absentes'},
                          linewidths=0.5, vmin=0)

    plt.title(f"Nombre de données absentes — {nom_colonne}", fontsize=16, pad=20)
    plt.xlabel('Date (année-mois)', fontsize=14)
    plt.ylabel('Région', fontsize=14)

    # Sauvegarde
    heatmap.figure.savefig(nom_fichier, bbox_inches='tight', dpi=300)
    plt.show()

    print(f"✅ Heatmap sauvegardée : {nom_fichier}")


In [ ]:
#  ******  ETUDE DES VALEURS EGALES A ZERO (0) ******

# Calcul du nombre de valeurs égales à 0 pour chaque colonne, par région et par mois.

count_zeros = df.groupby([df['region'], df['date_ym']]).apply(lambda x: (x==0).sum())

# Suppression des colonnes superflues de count_zeros

count_zeros = count_zeros.drop(count_zeros.columns[0:9], axis=1)

# Réinitialisation de l'index

count_zeros = count_zeros.reset_index()

# Export Excel du compte-rendu du nb de valeurs égales à 0 par colonne après regroupement Région/Jour-Mois

file_export_count_zeros ='Rapp_count_zeros_eco2mix.xlsx'
count_zeros.to_excel(file_export_count_zeros)
print("Rapport préliminaire de décompte des valeurs égales à 0 après regroupements exporté avec succès dans le fichier Excel 'Rapp_count_zeros_eco2mix.xlsx'")

# Export Excel du compte-rendu du nb de valeurs égales à 0 filtré

file_exp_count_zeros_filtre ='Rapp_count_zeros_eco2mix_FILTRE.xlsx'

count_zeros.to_excel(file_exp_count_zeros_filtre)

print("Rapport préliminaire de décompte des valeurs égales à 0 filtré exporté avec succès dans le fichier Excel 'Rapp_count_zeros_eco2mix_FILTRE.xlsx'")

# AFFICHAGE DES HEATMAPS DES VALEURS EGALES A 0, REGROUPEES PAR REGION ET PAR MOIS

# 1) Création d'un dictionnaire associant les colonnes mappées avec les noms des graphiques et des fichiers images à créer

dict_heatmaps_energie_zeros = {}
colonnes=['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
c=0
for i in range(2, 27):
    nom_colonne = count_zeros.columns[i]
    pivot = 'pivot_'+ nom_colonne[:7]
    nom_heatmap = nom_colonne[:9].replace(' ', '_') + 'zeros_map'
    nom_fichier = colonnes[c] +'_'+ nom_colonne[:9].replace(' ', '_') + '_zeros_heatmap.png'
    dict_heatmaps_energie_zeros[nom_colonne] = (pivot, nom_heatmap, nom_fichier)
    c+=1

display(dict_heatmaps_energie_zeros)

# 2) Affichage des chronogramme des valeurs égales à 0 pour chaque colonne de données d'énergie

for i in range(2,27):
    
    nom_colonne = count_zeros.columns[i]
    pivot, nom_heatmap, nom_fichier = dict_heatmaps_energie_zeros[nom_colonne]
                                                       
    # Pivot du DataFrame pour obtenir les dates comme colonnes
 
    pivot_df = count_zeros.pivot(index='region', columns='date_ym', values=nom_colonne)
    
    # Création d'un heatmap
    
    plt.figure(figsize=(100, 10))
    nom_heatmap = sns.heatmap(pivot_df, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Nombre de valeurs égales à 0'},
                                 linewidths = 1, vmin = 0, vmax = 1488)
    plt.title(f"Nombre de données égales à 0 pour la colonne {nom_colonne}, par région et mois", fontsize = 16, pad = 20)
    plt.xlabel('date_ym', fontsize = 16)
    plt.ylabel('region', fontsize = 16)
    plt.show()

    nom_heatmap.figure.savefig(nom_fichier)


In [ ]:
# Filtrage des colonnes numériques
selected_columns = [
    'consommation', 'thermique', 'nucleaire', 'eolien', 'solaire',
    'hydraulique', 'pompage', 'bioenergies', 'ech_physiques',
    'stockage_batterie', 'destockage_batterie',
    'tco_thermique', 'tch_thermique', 'tco_nucleaire', 'tch_nucleaire',
    'tco_eolien', 'tch_eolien', 'tco_solaire', 'tch_solaire',
    'tco_hydraulique', 'tch_hydraulique', 'tco_bioenergies', 'tch_bioenergies'
]
selected_columns_numeric = df[selected_columns].select_dtypes(include=['number']).columns

fig, axs = plt.subplots(13, 2, figsize=(15, 65))

# Aplatissement de l'array d'axes pour faciliter son utilisation
axs = axs.flatten()

plt.suptitle("Distributions des variables 'énergie' de Eco2Mix_Regional' brut")
# Création d'un histogramme pour chaque colonne numérique
for ax, col in zip(axs, selected_columns_numeric):
    sns.kdeplot(df[col], ax=ax)
    ax.set_title(col)

# Suppression de la zone de graphique inutilsée
fig.delaxes(axs[-1])

# Ajustement de l'espacement entre les sous-tracés
plt.tight_layout()
plt.subplots_adjust(top=0.95)

# Sauvegarde de la figure en haute résolution
plt.savefig('Histogrammes_données_brutes.png', dpi=300)

# Affichage de la figure
plt.show()

In [ ]:
# Affichage d'un heatmap des corrélations du fichier Eco2Mix Regional brut
colonnes_to_drop = ['code_insee_region', 'region', 'nature','date', 'mois','annee', 'date_ym','heure','date_heure']
df_reduce = df.drop(columns=colonnes_to_drop, axis = 1)

# Remplacement des chaînes '-' par NaN
df_reduce = df_reduce.replace('-', np.nan)

# Conversion en numérique forcée
df_reduce = df_reduce.apply(pd.to_numeric, errors='coerce')

cor = df_reduce.corr()
fig, axs = plt.subplots(figsize = (50,50))
savefighcor = sns.heatmap(cor, annot = True, ax = axs, cmap = "coolwarm")
plt.show()

# Exportation des graphiques dans des fichiers
savefighcor.figure.savefig('Heatmaps_corrélations_Eco2MixReg.png')